In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IA.utils import mapping, parameter_range
from scipy import stats
import math
from pathlib import Path

import sklearn
from sklearn.linear_model import LinearRegression
from sklearn import svm

from collections import defaultdict

In [ ]:
test_files = list(pd.read_csv("analysis/not_uploaded/AVA_test.csv")["file"])
gt = pd.read_csv("analysis/not_uploaded/AVA_gt.txt", sep=" ").drop(columns=["Unnamed: 0", "semanticTagID1", "semanticTagID2", "challengeID"])
df = pd.read_csv("analysis/not_uploaded/IA/AVA/.scratch.ckpts.IA.pexels.scores-one.change_class.epoch-5.pth.txt")
#df = pd.read_csv("analysis/not_uploaded/IA/AVA/.scratch.ckpts.IA.pexels.scores-one.change_regress.epoch-9.pth.txt")
#df = pd.read_csv("analysis/not_uploaded/IA/AVA/.scratch.ckpts.IA.pexels.scores-one.epoch-13.pth.txt")


df["img"] = df["img"].apply(lambda row: Path(row).stem)
df["score"] = df["score"].apply(lambda row: eval(row)[0])

gt["votes"] = gt.apply(lambda row: sum(list(row)[1:]), axis=1)
gt["gt_score"] = gt.apply(lambda row: sum([val * (i + 1) for i, val in enumerate(list(row)[1:-1])]), axis=1)
gt["gt_score"] = gt.apply(lambda row: row.gt_score / row.votes, axis=1)
gt["gt_quality"] = gt["gt_score"].apply(lambda row: 1 if row > 5 else 0)
gt = gt.drop(columns=[str(i) for i in range(1,11)] + ["votes"])

df["img"] = df["img"].astype(int)
gt["img"] = gt["img"].astype(int)

df = df.set_index("img")
gt = gt.set_index("img")

df = gt.join(df).dropna().drop_duplicates()
df["mode"] = df.index.isin(test_files)
df["mode"] = df["mode"].apply(lambda row: "test" if row else "train")

train_df = df[df["mode"] == "train"]
test_df = df[df["mode"] == "test"]

df

In [ ]:
stats.pearsonr(test_df["gt_score"], test_df["score"])

In [ ]:
stats.spearmanr(test_df["gt_score"], test_df["score"])

In [ ]:
sns.kdeplot(data=test_df, x="gt_score", y="score")

In [ ]:
test_df.columns

In [ ]:
features = [c for c in df.columns if c not in ["gt_score", "gt_quality", "mode"]]

In [ ]:
reg = svm.SVR()
reg.fit(train_df[features], train_df["gt_score"])
reg.score(test[features], test["gt_score"])

In [ ]:
reg.predict(test[features])

In [ ]:
sns.kdeplot(x=test_df["gt_score"], y=reg.predict(test[features]))